In [103]:
from ipyleaflet import (
    Map, GeoJSON, basemaps, basemap_to_tiles,
    LayersControl, SplitMapControl, DrawControl, MeasureControl
)
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
import json
import os

In [104]:
azero_geo_path = os.path.join('/','srv','data','azero','geo')
guadalquivir_geo_path = os.path.join('/','srv','data','guadalquivir','geo')

azero_geo_files = os.listdir(azero_geo_path)
guadalquivir_geo_files = os.listdir(guadalquivir_geo_path)

for files in [azero_geo_files, guadalquivir_geo_files]:
    checks = '.ipynb_checkpoints'
    if checks in files:
        files.remove(checks)

azero_layer_names = [file_name[:-5]for file_name in azero_geo_files]
guadalquivir_layer_names = [file_name[:-5]for file_name in guadalquivir_geo_files]

azero_layer_paths = [os.path.join(azero_geo_path, file_name + '.json') for file_name in azero_layer_names]
guadalquivir_layer_paths = [os.path.join(guadalquivir_geo_path, file_name + '.json') for file_name in guadalquivir_layer_names]

azero_name_to_path = dict(zip(azero_layer_names, azero_layer_paths))
guadalquivir_name_to_path = dict(zip(guadalquivir_layer_names, guadalquivir_layer_paths))

In [105]:
cuenca_toggle = widgets.ToggleButtons(
    options=['Azero', 'Guadalquivir',],
    value='Azero',
    tooltips=['Cuenca Azero', 'Cuenca Guadalquivir'],
)

layer_1_dropdown = widgets.Dropdown(
    options=azero_layer_names,
    description='Capa 1:',
)

color_1 = widgets.ColorPicker(
    concise=True,
    value='blue',
)

layer_2_dropdown = widgets.Dropdown(
    options=azero_layer_names,
    description='Capa 2:',
)
color_2 = widgets.ColorPicker(
    concise=True,
    value='green',
)

layer_3_dropdown = widgets.Dropdown(
    options=azero_layer_names,
    description='Capa 3:',
)
color_3 = widgets.ColorPicker(
    concise=True,
    value='yellow',
)

accordion = widgets.Accordion(children=[
    cuenca_toggle,
    widgets.VBox([
        widgets.HBox([layer_1_dropdown, color_1]),
        widgets.HBox([layer_2_dropdown, color_2]),
        widgets.HBox([layer_3_dropdown, color_3]),
    ])
])
accordion.set_title(0,'Cuenca')
accordion.set_title(1,'Capas')

go_button = widgets.Button(
    description='GO!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Mostrar capa',
)

my_map = Map(
    center=(-21.5119,-64.7253),
    zoom=7,
)
left_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
right_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
split_control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
my_map.add_control(split_control)
my_map.add_control(LayersControl())
my_map.add_control(DrawControl())
my_map.add_control(MeasureControl())

output_table = widgets.Output()

In [106]:
def handle_cuenca_change(change):
    if change.new == 'Azero':
        layer_1_dropdown.options = azero_layer_names
        layer_2_dropdown.options = azero_layer_names
        layer_3_dropdown.options = azero_layer_names
    elif change.new == 'Guadalquivir':
        layer_1_dropdown.options = guadalquivir_layer_names
        layer_2_dropdown.options = guadalquivir_layer_names
        layer_3_dropdown.options = guadalquivir_layer_names

def hover_handler(event=None, id=None, properties=None):
    output_table.clear_output()
    
    props = properties
    props.pop('style', None)
    
    with output_table:
        display(pd.DataFrame(props, index=[0]))

def handle_go_click(change):
    my_map.clear_layers()
    
    ln = [layer_1_dropdown.value, layer_2_dropdown.value, layer_3_dropdown.value]
    lc = [color_1.value, color_2.value, color_3.value]
    
    layer_names = []
    layer_colors = []
    for layer_name, layer_color in zip(ln, lc):
        if not layer_name in layer_names:
            layer_names.append(layer_name)
            layer_colors.append(layer_color)
        
    
    data_list = []
    
    for layer_name in layer_names:
        if cuenca_toggle.value == 'Azero':
            with open(azero_name_to_path[layer_name], 'r', encoding='utf-8') as f:
                data_list.append(json.load(f))
        elif cuenca_toggle.value == 'Guadalquivir':
            if layer_name == 'predios_proceso_saneamiento.json':
                with open(guadalquivir_name_to_path[layer_name], 'r') as f:
                    data_list.append(json.load(f))
            else:
                with open(guadalquivir_name_to_path[layer_name], 'r', encoding='utf-8') as f:
                    data_list.append(json.load(f))
        
    for data, color in zip(data_list, layer_colors):
        for feature in data['features']:
            feature['properties']['style'] = dict(
                color=color,
                weight=1,
                fillColor=color,
                fillOpacity=.3
            )
        
    selected_layers = [GeoJSON(
        data=data,
        name=layer_name,
        hover_style={'fillColor':'red'}
    ) for data, layer_name in zip(data_list, layer_names)]
    
    for selected_layer in selected_layers:
        my_map.add_layer(selected_layer)
        selected_layer.on_hover(hover_handler)
    

cuenca_toggle.observe(handle_cuenca_change, names='value')
go_button.on_click(handle_go_click)

In [107]:
widgets.VBox([accordion, go_button, my_map, output_table,])